In [16]:
!pip install ultralytics opencv-python 

DEPRECATION: pytorch-lightning 1.7.6 has a non-standard dependency specifier torch>=1.9.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: torchsde 0.2.5 has a non-standard dependency specifier numpy>=1.19.*; python_version >= "3.7". pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torchsde or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [1]:
import pycuda.autoinit
import pycuda.driver as cuda
print("CUDA Available:", torch.cuda.is_available())
print("Number of CUDA Devices:", torch.cuda.device_count())
for i in range(cuda.Device.count()):
    print(cuda.Device(i).name())
torch.cuda.set_device(0)
torch.cuda.empty_cache()

NameError: name 'torch' is not defined

In [1]:
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator

model = YOLO("../models/yolov8m-pose.pt")
import cv2
import torch
import time
def draw_keypoints(result, frame):
    annotator = Annotator(frame, line_width=1)
    for kps in result.keypoints:
        kps=kps.data.squeeze()
        annotator.kpts(kps)
        nkps=kps.cpu().numpy()
        for idx,(x,y,score) in enumerate(nkps):
            if score>0.5:
                cv2.circle(frame,(int(x),int(y)),3,(0,0,255),cv2.FILLED)
                cv2.putText(
                    frame,str(idx),(int(x),int(y)),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),1)
    return frame

def predict(frame, iou=0.7,conf=0.25):
    results=model(
        source=frame,
        device="cuda" if torch.cuda.is_available() else "cpu",
        #device="cpu",
        iou=0.7, #가중치
        conf=0.25, #
        verbose=False,
    )
    result = results[0]
    print(results[0])
    return result
def draw_boxes(result, frame):
    for boxes in result.boxes:
        x1,y1,x2,y2,score,classes=boxes.data.squeeze().cpu().numpy()
        cv2.rectangle(frame,(int(x1),int(y1)),(int(x2),int(y2)),(0,0,255),1)
    return frame

cap = cv2.VideoCapture(0)
fps=0
frame_count=0
start_time=time.time()
while True:
    ret, frame = cap.read()
    if not ret:
        print("웹캠에서 프레임을 읽을 수 없습니다.")
        break
    result=predict(frame)
    frame=draw_boxes(result,frame)
    frame=draw_keypoints(result,frame)
    frame_count += 1
    end_time = time.time()
    elapsed_time = end_time - start_time
    if elapsed_time >= 1:  # every second
        fps = frame_count / elapsed_time
        frame_count = 0
        start_time = time.time()
    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('Webcam', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
torch.cuda.empty_cache()

ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: ultralytics.engine.results.Keypoints object
masks: None
names: {0: 'person'}
obb: None
orig_img: array([[[128, 137, 157],
        [128, 136, 157],
        [137, 144, 166],
        ...,
        [154, 151, 166],
        [156, 153, 167],
        [158, 155, 169]],

       [[131, 138, 160],
        [130, 137, 157],
        [130, 138, 155],
        ...,
        [154, 153, 163],
        [155, 152, 164],
        [158, 156, 167]],

       [[136, 143, 163],
        [131, 139, 157],
        [130, 138, 155],
        ...,
        [154, 154, 163],
        [154, 154, 163],
        [157, 158, 167]],

       ...,

       [[189, 198, 200],
        [195, 204, 205],
        [183, 192, 194],
        ...,
        [249, 249, 249],
        [248, 248, 248],
        [249, 249, 249]],

       [[180, 191, 193],
        [184, 194, 195],
        [180, 189, 190],
        ...,
        [249, 249, 249],

In [2]:
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator

model = YOLO("../models/yolov8s-pose.pt")
import cv2
import torch
import time
def draw_keypoints(result, frame):
    annotator = Annotator(frame, line_width=1)
    for kps in result.keypoints:
        kps=kps.data.squeeze()
        annotator.kpts(kps)
        nkps=kps.cpu().numpy()
        for idx,(x,y,score) in enumerate(nkps):
            if score>0.5:
                cv2.circle(frame,(int(x),int(y)),3,(0,0,255),cv2.FILLED)
                cv2.putText(
                    frame,str(idx),(int(x),int(y)),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),1)
    return frame

def predict(frame, iou=0.7,conf=0.25):
    results=model(
        source=frame,
        device="cuda" if torch.cuda.is_available() else "cpu",
        #device="cpu",
        iou=0.7, #가중치
        conf=0.25, #
        verbose=False,
    )
    result = results[0]
    return result
def draw_boxes(result, frame):
    for boxes in result.boxes:
        x1,y1,x2,y2,score,classes=boxes.data.squeeze().cpu().numpy()
        cv2.rectangle(frame,(int(x1),int(y1)),(int(x2),int(y2)),(0,0,255),1)
    return frame

cap = cv2.VideoCapture(r"C:\Users\z\OneDrive - Sejong University\바탕 화면\2024년 1학기 공부\yolo학습용데이터\5.mp4")
fps=0
frame_count=0
start_time=time.time()
while True:
    ret, frame = cap.read()
    if not ret:
        print("웹캠에서 프레임을 읽을 수 없습니다.")
        break
    result=predict(frame)
    frame=draw_boxes(result,frame)
    frame=draw_keypoints(result,frame)
    frame_count += 1
    end_time = time.time()
    elapsed_time = end_time - start_time
    if elapsed_time >= 1:  # every second
        fps = frame_count / elapsed_time
        frame_count = 0
        start_time = time.time()
    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('Webcam', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
torch.cuda.empty_cache()

In [ ]:
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator

model = YOLO("../models/yolov8m.pt")
import cv2
import torch
import time
def predict(frame, iou=0.7,conf=0.25):
    results=model(
        source=frame,
        device="cuda" if torch.cuda.is_available() else "cpu",
        #device="cpu",
        iou=0.9, #가중치
        conf=0.25, #
        verbose=False,
    )
    result = results[0]
    #print(results[0])
    return result
def draw_boxes(result, frame):
    for boxes in result.boxes:
        if boxes.id!=0:
            x1,y1,x2,y2,score,classes=boxes.data.squeeze().cpu().numpy()
            cv2.rectangle(frame,(int(x1),int(y1)),(int(x2),int(y2)),(0,0,255),1)
            class_name = result.names[classes]
            label = f"{class_name}: {score:.2f}"
            cv2.putText(frame, label, (int(x1), int(y1-10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2)
        
    return frame

cap = cv2.VideoCapture(r"C:\Users\z\OneDrive - Sejong University\바탕 화면\2024년 1학기 공부\yolo학습용데이터\6.mp4")
fps=0
frame_count=0
start_time=time.time()
while True:
    ret, frame = cap.read()
    if not ret:
        print("웹캠에서 프레임을 읽을 수 없습니다.")
        break
    result=predict(frame)
    frame=draw_boxes(result,frame)
    frame_count += 1
    end_time = time.time()
    elapsed_time = end_time - start_time
    if elapsed_time >= 1:  # every second
        fps = frame_count / elapsed_time
        frame_count = 0
        start_time = time.time()
    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('Webcam', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
torch.cuda.empty_cache()